# What celebration is it?

Install fastai and duckduckgo

In [ ]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search

Prepare code to be exported

In [ ]:
#|default_exp app

## Download images of christmas, easter, and ramadan celebration

Create search function

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=100):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

Test the search and view the image result

In [ ]:
#NB: `search_images` depends on duckduckgo.com, which doesn't always return correct responses.
#    If you get a JSON error, just try running it again (it may take a couple of tries).
urls = search_images('christmas celebration photos', max_images=1)
urls[0]

Download **christmas celebration photos** and convert image to thumbnail size

In [ ]:
from fastdownload import download_url
dest = 'christmas.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

Do the same for **easter celebration photos**

In [ ]:
download_url(search_images('easter celebration photos', max_images=1)[0], 'easter.jpg', show_progress=False)
Image.open('easter.jpg').to_thumb(256,256)

Also do the same for **ramadan celebration photos**

In [ ]:
download_url(search_images('ramadan celebration photos', max_images=1)[0], 'ramadan.jpg', show_progress=False)
Image.open('ramadan.jpg').to_thumb(256,256)

Grab few examples of christmas, easter, and ramadan photos from the web and save into a different folder

In [ ]:
searches = 'christmas celebration','easter celebration', 'ramadan celebration'
path = Path('celebration')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

# Train the model

Remove images that failed to download correctly

In [ ]:
#|export
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

Prepare to train the model with `DataLoaders`, which contains a training set and a validation set. And then view a sample image from it. 

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=9)

Train the model with the computer vision model, `resnet18`. And `fine_tune()` it.

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

# Use the model

Test the model with an image to see how well it does with the prediction after training

In [ ]:
what_celebration,_,probs = learn.predict(PILImage.create('christmas.jpg'))
print(f"This is a: {what_celebration}.")
print(f"Probability it's christmas: {probs[0]:.4f}")

# Save the model

Export the trained `Learner`. This contains all the information needed to run the model.

In [ ]:
learn.export('model.pkl')